In [10]:
# Import required libraries
import ikpy.chain
import numpy as np
import time
import math
import plotly.graph_objects as go
import ipywidgets as widgets
#import serial


In [11]:
# Load the robot chain from a URDF file
my_chain = ikpy.chain.Chain.from_urdf_file("arm_urdf.urdf", active_links_mask=[False, True, True, True, True, True, True])

In [12]:
# Define initial target position and orientation
target_position = [0, 0, 0.58]
target_orientation = [-1, 0, 0]

In [13]:
# Function to perform inverse kinematics
def compute_ik(target_position, target_orientation, orientation_mode="Y"):
    ik = my_chain.inverse_kinematics(target_position, target_orientation, orientation_mode=orientation_mode)
    return ik

In [14]:
# Initialize Plotly plot for real-time visualization
fig = go.FigureWidget()
fig.add_scatter3d(x=[], y=[], z=[], mode='lines+markers')

# Function to update the plot
def update_plot(ik):
    positions = my_chain.forward_kinematics(ik)[:3, 3]
    fig.data[0].x = [positions[0]]
    fig.data[0].y = [positions[1]]
    fig.data[0].z = [positions[2]]
    fig.update_layout(scene=dict(xaxis=dict(range=[-0.5, 0.5]), yaxis=dict(range=[-0.5, 0.5]), zaxis=dict(range=[0, 0.6])))

In [15]:
# Move function updates target position, computes IK, and updates the plot
def move(x, y, z):
    global target_position
    target_position = [x, y, z]
    ik = compute_ik(target_position, target_orientation)
    update_plot(ik)

In [16]:
# Optional: Initialize serial communication (you can adjust the port and parameters)
#ser = serial.Serial('COM3', 9600, timeout=1)

# Function to send commands via serial
# def send_command(ik, move_time=1):
#     angles = list(map(lambda r: math.degrees(r), ik.tolist()))
#     command = ' '.join(f'{i}:{angle:.2f}' for i, angle in enumerate(angles)) + f' t{move_time}\n'
#     ser.write(command.encode('ASCII'))

In [17]:
# Controller setup (put at the end and easy to comment out)
# async def control_loop():
#     con = widgets.Controller()
#     display(con)
#     x, y, z = 0, 0.25, 0.1
#     while con.buttons[9].value < 1:
#         xp, yp, zp = con.axes[0].value, con.axes[1].value, con.axes[2].value
#         if abs(xp) > 0.1 or abs(yp) > 0.1 or abs(zp) > 0.1:
#             x += xp / 100
#             y -= yp / 100
#             z -= zp / 100
#             move(x, y, z)
#         await asyncio.sleep(0.05)

# loop = asyncio.get_event_loop()
# loop.create_task(control_loop())

In [23]:
import ikpy.chain
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interactive, HBox, VBox, FloatSlider

# Load the robot chain from a URDF file
my_chain = ikpy.chain.Chain.from_urdf_file("arm_urdf.urdf", active_links_mask=[False, True, True, True, True, True, True])

# Define initial target position and orientation
initial_position = [0, 0.2, 0.3]
target_orientation = [-1, 0, 0]  # Assuming the orientation doesn't change for simplicity

# Initialize Plotly plot for real-time visualization
fig = go.FigureWidget()
scatter = fig.add_scatter3d(x=[], y=[], z=[], mode='lines+markers')
fig.update_layout(scene=dict(
                    xaxis=dict(range=[-0.5, 0.5], autorange=False),
                    yaxis=dict(range=[-0.5, 0.5], autorange=False),
                    zaxis=dict(range=[0, 1.0], autorange=False)),
                    width=700,
                    margin=dict(r=20, l=10, b=10, t=10))


# Function to compute and plot the IK results
def update_plot(x, y, z):
    target_position = [x, y, z]
    ik = my_chain.inverse_kinematics(target_position, target_orientation)
    positions = my_chain.forward_kinematics(ik)[:3, 3]
    if positions.ndim == 1:  # Check if positions is a 1D array, meaning only one point
        positions = np.array([positions])  # Convert to a 2D array with one row
    with fig.batch_update():
        fig.data[0].update(x=positions[:, 0].tolist(), y=positions[:, 1].tolist(), z=positions[:, 2].tolist())


# Sliders to control the target position
x_slider = FloatSlider(value=initial_position[0], min=-0.5, max=0.5, step=0.01, description='X')
y_slider = FloatSlider(value=initial_position[1], min=-0.5, max=0.5, step=0.01, description='Y')
z_slider = FloatSlider(value=initial_position[2], min=0.0, max=1.0, step=0.01, description='Z')

# Function to update the plot whenever the sliders change
def on_value_change(change):
    update_plot(x_slider.value, y_slider.value, z_slider.value)

# Watch for slider changes
x_slider.observe(on_value_change, names='value')
y_slider.observe(on_value_change, names='value')
z_slider.observe(on_value_change, names='value')

# Layout widgets
sliders = VBox([x_slider, y_slider, z_slider])
ui = VBox([sliders, fig])

# Display UI
display(ui)
